In [ ]:
import os
import glob
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import pydicom as dicom
import tensorflow as tf
from keras import layers
from pydicom import dcmread
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import requests
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from keras_preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda,Dense,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
kernel_initializer =  'he_uniform'

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train_b = pd.read_csv("train_bounding_boxes.csv")
submission = pd.read_csv("sample_submission.csv")

In [ ]:
train.sort_values('StudyInstanceUID',inplace=True,axis = 0)
train_b.sort_values('StudyInstanceUID',inplace=True,axis = 0)
train.reset_index(inplace=True,drop= True)
train_b.reset_index(inplace=True,drop= True)

In [ ]:
patients = sorted(os.listdir(r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"))

In [ ]:
train_b['startX']=0.0
train_b['startY']=0.0
train_b['endX']=0.0
train_b['endY']=0.0
for i in range(len(train_b)):
    # calculate top,left for box
    train_b['startX'][i] = train_b['x'][i] - (train_b['width'][i]/2)
    train_b['startY'][i] = train_b['y'][i] + (train_b['height'][i]/2)
    # calculate right,bottom for box
    train_b['endX'][i] = train_b['x'][i] + (train_b['width'][i]/2)
    train_b['endY'][i] = train_b['y'][i] - (train_b['height'][i]/2)

In [ ]:
# for i in range(len(train)):
#     dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
#     for j in range(len(train_b)):
#         if train['StudyInstanceUID'][i] == train_b['StudyInstanceUID'][j]:
#             sec = str(train_b['slice_number'][j]) +".dcm"
#             image_path= os.path.abspath(os.path.join(dir,sec))
#             ds = dicom.dcmread(image_path)
#             pixel_array_numpy = ds.pixel_array
#             if pixel_array_numpy.shape != (512,512):
#                 print(pixel_array_numpy.shape)

In [ ]:
data =[]
targets = []
filenames = []
trainlabel = []
for i in range(len(train)):
    dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
    for j in range(len(train_b)):
        if train['StudyInstanceUID'][i] == train_b['StudyInstanceUID'][j]:
            sec = str(train_b['slice_number'][j]) +".dcm"
            image_path= os.path.abspath(os.path.join(dir,sec))
            ds = dicom.dcmread(image_path)
            pixel_array_numpy = ds.pixel_array
            if pixel_array_numpy.shape != (512,512):
                pixel_array_numpy =cv.resize(pixel_array_numpy, (512,512))
            data.append(pixel_array_numpy/-255)
            targets.append([train_b['startX'][j], train_b['startY'][j], train_b['endX'][j], train_b['endY'][j]])
            filenames.append(train_b['StudyInstanceUID'][j])
            cur_label = []
            cur_label.append(train.loc[i,'C1'])
            cur_label.append(train.loc[i,'C2'])
            cur_label.append(train.loc[i,'C3'])
            cur_label.append(train.loc[i,'C4'])
            cur_label.append(train.loc[i,'C5'])
            cur_label.append(train.loc[i,'C6'])
            cur_label.append(train.loc[i,'C7'])
            trainlabel += [cur_label]
            print(trainlabel)

In [ ]:
def get_model():
    inputs = Input((512, 512 ,1))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    
    #Expansive path 
    # u6 = Conv2DTranspose(128, (2, 2), strides=2, padding='same')(c5)
    # u6 = concatenate([u6, c4])
    # c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    # c6 = Dropout(0.2)(c6)
    # c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)
     
    # u7 = Conv2DTranspose(64, (2, 2), strides=2, padding='same')(c6)
    # u7 = concatenate([u7, c3])
    # c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    # c7 = Dropout(0.2)(c7)
    # c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)
     
    # u8 = Conv2DTranspose(32, (2, 2), strides=2, padding='same')(c7)
    # u8 = concatenate([u8, c2])
    # c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    # c8 = Dropout(0.1)(c8)
    # c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)
     
    # u9 = Conv2DTranspose(16, (2, 2), strides=2, padding='same')(c8)
    # u9 = concatenate([u9, c1])
    # c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    # c9 = Dropout(0.1)(c9)
    # c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)
    f1 = Flatten()(p3)
    d1 =  Dense(128, activation='relu')(f1)
    d2 =  Dense(64, activation='relu')(d1)
    d3 =  Dense(32, activation='relu')(d2)
    outputs = Dense(4, activation='softmax')(d3)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    #compile model outside of this function to make it flexible. 

    return model

In [ ]:
model = get_model()
model.compile(optimizer = "Adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
y = np.array(targets)

In [ ]:
y.shape

In [ ]:
targets

In [ ]:
type(data)

In [ ]:
data[0].shape

In [ ]:
data[1].shape

In [ ]:
x= np.array(data)

In [ ]:
x.shape

In [ ]:
x = np.asarray(x).astype('float32')

In [ ]:
type(x)

In [ ]:
from sklearn.model_selection import train_test_split
split = train_test_split(x, y,trainlabel, test_size=0.10,
	random_state=0)
(trainImages, valImages) = split[:2]
(trainTargets, valTargets) = split[2:4]

(trainFilenames, valFilenames) = split[4:]


In [ ]:
H = model.fit(
	trainImages, trainTargets,
	validation_data=0.1,
	batch_size=64,
	epochs=3,
	verbose=1)

In [ ]:
y.shape

In [ ]:
x.shape

In [ ]:
model.predict(x[:3]).shape